In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image
import glob as glob
import sys

In [ ]:
# faster rcnn inception v2
!gdown https://drive.google.com/u/0/uc?id=1LRCSWIkX_i6ijScMfaxSte_5a_x9tjWF&export=download
!unzip faster_rcnn_inception_v2.zip -d models/faster_rcnn_inception_v2

In [ ]:
# ssd_inception_v2
!gdown https://drive.google.com/u/0/uc?id=1TKMd-wIZJ1aUcOhWburm2b6WgYnP0ZK6&export=download
!unzip ssd_inception_v2.zip -d models/ssd_inception_v2

In [ ]:
# ssd_mobilenet_v1
!gdown https://drive.google.com/u/0/uc?id=1U31RhUvE1Urr5Q92AJynMvl-oFBVRxxg&export=download
!unzip ssd_mobilenet_v1.zip -d models/ssd_mobilenet_v1

In [ ]:
# faster_rcnn_resnet_101
!gdown https://drive.google.com/u/0/uc?id=15OxyPlqyOOlUdsbUmdrexKLpHy1l5tP9&export=download
!unzip faster_rcnn_resnet_101.zip -d models/faster_rcnn_resnet_101

In [ ]:
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous'
# MODEL_NAME = 'faster_rcnn_resnet_101'
# MODEL_NAME = 'faster_rcnn_resnet50'
MODEL_NAME = 'faster_rcnn_inception_v2'
# MODEL_NAME = 'rfcn_resnet101'
# MODEL_NAME = 'ssd_inception_v2'
# MODEL_NAME = 'ssd_mobilenet_v1'

MODEL_PATH = os.path.join('models', MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'inference_graph/frozen_inference_graph.pb')

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [ ]:
import time

image_path = 'image.ppm'
image = Image.open(image_path)

with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:

        times = []
        for i in range(5):
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')

            start = time.time()
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            end = time.time()
            times.append(end - start)
            
        print(np.mean(times))

1.1142212867736816
